In [1]:
import hist
import boost_histogram as bh
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt
from wremnants import boostHistHelpers as hh
from wremnants import histselections as sel
from wremnants import datasets2016
from wremnants import plot_tools
import wremnants
import lz4.frame
import pickle
hep.style.use(hep.style.ROOT)

Welcome to JupyROOT 6.27/01


In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
with lz4.frame.open("../w_z_gen_dists_reweight_h2.pkl.lz4") as f:
    file = pickle.load(f)
file['WplusmunuPostVFP'        ]["output"]["helicity_moments_scale"]

Hist(
  Variable([0, inf], name='massVgen', label='massVgen'),
  Variable([0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 10], name='absYVgen', label='absYVgen'),
  Variable([0, 2, 3, 4, 4.75, 5.5, 6.5, 8, 9, 10, 12, 14, 16, 18, 20, 23, 27, 32, 40, 55, 100], name='ptVgen', label='ptVgen'),
  Regular(2, -2, 2, underflow=False, overflow=False, name='chargeVgen', label='chargeVgen'),
  storage=Weight()) # Sum: WeightedSum(value=4.08032e+08, variance=4.08032e+08) (WeightedSum(value=4.1291e+08, variance=4.1291e+08) with flow)

In [4]:
hist_ang_coeff = {}
sigma_UL = {}
boson_channels = ['wp', 'wm', 'z']
projection_axes = ['ptVgen', 'absYVgen']
for chn in boson_channels:
    hist_ang_coeff[chn] = {}
    sigma_UL[chn] = {}

# read full histogram for angular coefficients
hist_ang_coeff['wp']['bugged'] = file['WplusmunuPostVFP'        ]['output']['helicity_moments_scale']
hist_ang_coeff['wp']['bugfix'] = file['WplusmunuPostVFP_bugfix' ]['output']['helicity_moments_scale']
hist_ang_coeff['wm']['bugged'] = file['WminusmunuPostVFP'       ]['output']['helicity_moments_scale']
hist_ang_coeff['wm']['bugfix'] = file['WminusmunuPostVFP_bugfix']['output']['helicity_moments_scale']
hist_ang_coeff['z' ]['bugged'] = file['ZmumuPostVFP'            ]['output']['helicity_moments_scale']
hist_ang_coeff['z' ]['bugfix'] = file['ZmumuPostVFP_bugfix'     ]['output']['helicity_moments_scale']

s = hist.tag.Slicer()
sigma_UL['wp']['bugged'] = (file['WplusmunuPostVFP'        ]['output']['nominal_gen']* file["WplusmunuPostVFP"        ]["dataset"]["xsec"] / file["WplusmunuPostVFP"        ]["weight_sum"]).project('ptVgen')
sigma_UL['wp']['bugfix'] = (file['WplusmunuPostVFP_bugfix' ]['output']['nominal_gen']* file["WplusmunuPostVFP_bugfix" ]["dataset"]["xsec"] / file["WplusmunuPostVFP_bugfix" ]["weight_sum"]).project('ptVgen')
sigma_UL['wm']['bugged'] = (file['WminusmunuPostVFP'       ]['output']['nominal_gen']* file["WminusmunuPostVFP"       ]["dataset"]["xsec"] / file["WminusmunuPostVFP"       ]["weight_sum"]).project('ptVgen')
sigma_UL['wm']['bugfix'] = (file['WminusmunuPostVFP_bugfix']['output']['nominal_gen']* file["WminusmunuPostVFP_bugfix"]["dataset"]["xsec"] / file["WminusmunuPostVFP_bugfix"]["weight_sum"]).project('ptVgen')
sigma_UL['z' ]['bugged'] = (file['ZmumuPostVFP'            ]['output']['nominal_gen'][{'massVgen':s[80j:100j:hist.sum]}] * file["ZmumuPostVFP"]["dataset"]["xsec"] / file["ZmumuPostVFP"]["weight_sum"]).project('ptVgen')
sigma_UL['z' ]['bugfix'] = (file['ZmumuPostVFP_bugfix'     ]['output']['nominal_gen'][{'massVgen':s[80j:100j:hist.sum]}] * file["ZmumuPostVFP_bugfix"]["dataset"]["xsec"] / file["ZmumuPostVFP_bugfix"]["weight_sum"]).project('ptVgen')
# make projected 1D histograms; "normalize" and calculate the angular coefficients 
hist1D_ang_coeff = {}
s = hist.tag.Slicer()
for chn in boson_channels:
    hist1D_ang_coeff[chn] = {}
    for group in ['bugged', 'bugfix']:
        hist1D_ang_coeff[chn][group] = {}
        for ax in projection_axes:
            if chn == 'z':
                hist1D_ang_coeff[chn][group][ax] = wremnants.moments_to_angular_coeffs(hist_ang_coeff[chn][group][{'massVgen':s[80j:100j:hist.sum]}].project('helicity',ax))
            else:
                hist1D_ang_coeff[chn][group][ax] = wremnants.moments_to_angular_coeffs(hist_ang_coeff[chn][group].project('helicity',ax))


ValueError: The axis name helicity could not be found

In [ ]:
'''make plots for angular coefficients in the following scheme'''
#                Z
#-----------ptV A_0-7  ----------
#-----------etaV A_0-7 ----------
#                W+
#-----------ptV A_0-7  ----------
#-----------etaV A_0-7 ----------
#              W-
#-----------ptV A_0-7  ----------
#-----------etaV A_0-7 ----------
'''
fig, axs = plt.subplots(6, 9, figsize=(36,54))

for idx_ang in range(-1,8):
    for idx_ax, ax in enumerate(projection_axes):
         plot_tools.makePlotWithRatioToRef(
            [hist1D_ang_coeff['z']['bugged'][ax][{"helicity":idx_ang}], 
             hist1D_ang_coeff['z']['bugfix'][ax][{"helicity":idx_ang}]], 
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel=ax, 
            ylabel="Events/bin",
            rlabel="bugfix/bugged",
            xlim=None, binwnorm=None).plot(axs[0+idx_ax, idx_ang+1])

'''

In [ ]:
rrange_opt = None

for idx_ang in range(-1,8):
    if idx_ang == -1:
        ang_name = "Const_term"
    else:
        ang_name = f"A_{idx_ang}"
    
    rrange_opt = [0.9,1.1]
    if idx_ang == 0:
        rrange_opt = [0.8,1.2]

    for idx_ax, ax in enumerate(projection_axes):
        fig_z = plot_tools.makePlotWithRatioToRef(
            [hist1D_ang_coeff['z']['bugged'][ax][{"helicity":hist.loc(idx_ang)}], 
             hist1D_ang_coeff['z']['bugfix'][ax][{"helicity":hist.loc(idx_ang)}]], 
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel=ax, 
            ylabel="Events/bin",
            rlabel="bugfix/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)
        fig_wp = plot_tools.makePlotWithRatioToRef(
            [hist1D_ang_coeff['wp']['bugged'][ax][{"helicity":hist.loc(idx_ang)}], 
             hist1D_ang_coeff['wp']['bugfix'][ax][{"helicity":hist.loc(idx_ang)}]], 
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel=ax, 
            ylabel="Events/bin",
            rlabel="bugfix/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)
        fig_wm = plot_tools.makePlotWithRatioToRef(
            [hist1D_ang_coeff['wm']['bugged'][ax][{"helicity":hist.loc(idx_ang)}], 
             hist1D_ang_coeff['wm']['bugfix'][ax][{"helicity":hist.loc(idx_ang)}]], 
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel=ax, 
            ylabel="Events/bin",
            rlabel="bugfix/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)
        fig_z.savefig(f"ang_coeff-{ang_name}-{ax}-Z.pdf")
        fig_wp.savefig(f"ang_coeff-{ang_name}-{ax}-wp.pdf")
        fig_wm.savefig(f"ang_coeff-{ang_name}-{ax}-wm.pdf")
        fig_z.savefig(f"ang_coeff-{ang_name}-{ax}-Z.png")
        fig_wp.savefig(f"ang_coeff-{ang_name}-{ax}-wp.png")
        fig_wm.savefig(f"ang_coeff-{ang_name}-{ax}-wm.png")

In [ ]:
import uproot
f = uproot.open("/eos/user/k/kelong/HistFiles/ZGen/ZToMuMu_MATRIX_RadISH_MatchEWParams_NNPDF31.root")
hist_corr = f["DYm50_matrix__radish/ptZ_lhe_mm"].to_hist()

with lz4.frame.open("../w_z_gen_dists_fine_bin.pkl.lz4") as f:
    minnlo = pickle.load(f)
    
hist_bugged = (minnlo["ZmumuPostVFP"]["output"]["nominal_gen"] *
               minnlo["ZmumuPostVFP"]["dataset"]["xsec"] / minnlo["ZmumuPostVFP"]["weight_sum"]).project('ptVgen')
hist_bugfix = (minnlo["ZmumuPostVFP_bugfix"]["output"]["nominal_gen"] *
               minnlo["ZmumuPostVFP_bugfix"]["dataset"]["xsec"] / minnlo["ZmumuPostVFP_bugfix"]["weight_sum"]).project('ptVgen')

plot_tools.makePlotWithRatioToRef(
            [hist_corr, 
             hist_bugged,
             hist_bugfix], 
            ["MATRIX+RadISH (NNLO+N$^{3}$LL)", "MiNNLO (H$^{(2)}$ sign error", "MiNNLO (H$^{(2)}$ sign error fixed)"], 

            colors=['blue', 'green', 'orange'], 
            xlabel="$p_T^Z$ (GeV)", 
            ylabel="$\sigma$/bin",
            rlabel="x/NNLO+N$^{3}$LL",
            rrange=[0.9, 1.1],
            xlim=None, binwnorm=None, baseline=True)

In [ ]:
file['WplusmunuPostVFP_bugfix' ]['output']['nominal_gen'].project('ptVgen')

In [ ]:
fig = plot_tools.makePlotWithRatioToRef(
            [hist1D_ang_coeff['z']['bugged']['ptVgen'][{"helicity":4j}], 
             hist1D_ang_coeff['z']['bugfix']['ptVgen'][{"helicity":4j}]],
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel='ptVgen', 
            ylabel="$A_4$",
            rlabel="bugfix/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)

fig = plot_tools.makePlotWithRatioToRef(
            [sigma_UL['z']['bugged'], 
             sigma_UL['z']['bugfix']],
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel='ptVgen', 
            ylabel="$\sigma_{UL}$",
            rlabel="bugfix/bugged",
            rrange=[0.95,1.05],
            xlim=None, binwnorm=None, baseline=True)

fig = plot_tools.makePlotWithRatioToRef(
            [hh.multiplyHists(hist1D_ang_coeff['z']['bugged']['ptVgen'][{"helicity":4j}], sigma_UL['z']['bugged'], allowBroadcast=True), 
             hh.multiplyHists(hist1D_ang_coeff['z']['bugfix']['ptVgen'][{"helicity":4j}], sigma_UL['z']['bugfix'], allowBroadcast=True)],
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel='ptVgen', 
            ylabel="$A_4 * \sigma _{UL}$",
            rlabel="bugfix/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)

In [ ]:
fig = plot_tools.makePlotWithRatioToRef(
            [hist1D_ang_coeff['wp']['bugged']['ptVgen'][{"helicity":4j}], 
             hist1D_ang_coeff['wp']['bugfix']['ptVgen'][{"helicity":4j}]],
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel='$p_T^{W^+}$', 
            ylabel="$A_4$",
            rlabel="bugfix/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)

fig = plot_tools.makePlotWithRatioToRef(
            [sigma_UL['wp']['bugged'], 
             sigma_UL['wp']['bugfix']],
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel='$p_T^{W^+}$', 
            ylabel="$\sigma_{UL}$",
            rlabel="bugfix/bugged",
            rrange=[0.95,1.05],
            xlim=None, binwnorm=None, baseline=True)

fig = plot_tools.makePlotWithRatioToRef(
            [hh.multiplyHists(hist1D_ang_coeff['wp']['bugged']['ptVgen'][{"helicity":4j}], sigma_UL['wp']['bugged'], allowBroadcast=True), 
             hh.multiplyHists(hist1D_ang_coeff['wp']['bugfix']['ptVgen'][{"helicity":4j}], sigma_UL['wp']['bugfix'], allowBroadcast=True)],
            ["bugged", 'bugfix'], 
            colors=['black', 'red'], 
            xlabel='$p_T^{W^+}$', 
            ylabel="$A_4 * \sigma _{UL}$",
            rlabel="bugfix/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)

In [ ]:
with lz4.frame.open("../w_z_gen_dists_slc7.pkl.lz4") as f:
    file_slc7 = pickle.load(f)
hist_ang_coeff = {}
sigma_UL = {}
boson_channels = ['wm']
projection_axes = ['ptVgen', 'absYVgen']
for chn in boson_channels:
    hist_ang_coeff[chn] = {}
    sigma_UL[chn] = {}

# read full histogram for angular coefficients

hist_ang_coeff['wm']['bugged'     ] = file_slc7['WminusmunuPostVFP'            ]['output']['helicity_moments_scale']
hist_ang_coeff['wm']['bugfix'     ] = file_slc7['WminusmunuPostVFP_bugfix'     ]['output']['helicity_moments_scale']
hist_ang_coeff['wm']['bugfix_slc7'] = file_slc7['WminusmunuPostVFP_bugfix_slc7']['output']['helicity_moments_scale']



sigma_UL['wm']['bugged'     ] = (file_slc7['WminusmunuPostVFP'            ]['output']['nominal_gen']* file_slc7["WminusmunuPostVFP"            ]["dataset"]["xsec"] / file_slc7["WminusmunuPostVFP"            ]["weight_sum"]).project('ptVgen')
sigma_UL['wm']['bugfix'     ] = (file_slc7['WminusmunuPostVFP_bugfix'     ]['output']['nominal_gen']* file_slc7["WminusmunuPostVFP_bugfix"     ]["dataset"]["xsec"] / file_slc7["WminusmunuPostVFP_bugfix"     ]["weight_sum"]).project('ptVgen')
sigma_UL['wm']['bugfix_slc7'] = (file_slc7['WminusmunuPostVFP_bugfix_slc7']['output']['nominal_gen']* file_slc7["WminusmunuPostVFP_bugfix_slc7"]["dataset"]["xsec"] / file_slc7["WminusmunuPostVFP_bugfix_slc7"]["weight_sum"]).project('ptVgen')

# make projected 1D histograms; "normalize" and calculate the angular coefficients 
hist1D_ang_coeff = {}
s = hist.tag.Slicer()
for chn in boson_channels:
    hist1D_ang_coeff[chn] = {}
    for group in ['bugged', 'bugfix', 'bugfix_slc7']:
        hist1D_ang_coeff[chn][group] = {}
        for ax in projection_axes:
            if chn == 'z':
                hist1D_ang_coeff[chn][group][ax] = wremnants.moments_to_angular_coeffs(hist_ang_coeff[chn][group][{'massVgen':s[80j:100j:hist.sum]}].project('helicity',ax))
            else:
                hist1D_ang_coeff[chn][group][ax] = wremnants.moments_to_angular_coeffs(hist_ang_coeff[chn][group].project('helicity',ax))


In [ ]:
rrange_opt = None

for idx_ang in range(-1,8):
    if idx_ang == -1:
        ang_name = "Const_term"
    else:
        ang_name = f"A_{idx_ang}"
    
    rrange_opt = [0.9,1.1]
    if idx_ang == 0:
        rrange_opt = [0.8,1.2]

    for idx_ax, ax in enumerate(projection_axes):
        fig_wm = plot_tools.makePlotWithRatioToRef(
            [hist1D_ang_coeff['wm']['bugged'][ax][{"helicity":hist.loc(idx_ang)}], 
             hist1D_ang_coeff['wm']['bugfix'][ax][{"helicity":hist.loc(idx_ang)}],
             hist1D_ang_coeff['wm']['bugfix_slc7'][ax][{"helicity":hist.loc(idx_ang)}]], 
            ["bugged", 'bugfix', 'bugfix_slc7'], 
            colors=['black', 'red', 'green'], 
            xlabel=ax, 
            ylabel="Events/bin",
            rlabel="bugfixes/bugged",
            rrange=rrange_opt,
            xlim=None, binwnorm=None, baseline=True)

        fig_wm.savefig(f"ang_coeff-{ang_name}-{ax}-wm.pdf")
        fig_wm.savefig(f"ang_coeff-{ang_name}-{ax}-wm.png")